In [1]:
# Cell 1: Import libraries for data handling, random data, and PDF creation
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
import json
import ollama
from pydantic import BaseModel, Field
from typing import List
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
import os

# Initialize Faker for fake data
fake = Faker()


In [2]:
# Cell 2: Create directory


In [3]:

# Cell 3: Pydantic structured output for a transaction
class Transaction(BaseModel):
    description: str = Field(..., max_length=35, description="Transaction description, max 35 characters")
    category: str
    amount: float


In [4]:
# Cell 4: Generate category lists using LLM
def generate_category_lists() -> tuple[List[str], List[str]]:
    prompt = """
    Generate two lists of bank transaction categories in JSON format. One list for reasons someone loses money (e.g., utilities, subscriptions) and one for reasons someone gains money (e.g., gifts, deposits). Each list should have 5-7 unique categories, each 1-2 words, title case, no punctuation. Return as:
    {
      "loss_categories": ["Category One", "Category Two", ...],
      "gain_categories": ["Category One", "Category Two", ...]
    }
    """
    try:
        response = ollama.generate(model="mistral:7b-instruct-v0.3-q4_0", prompt=prompt)
        category_data = json.loads(response['response'].strip())
        loss_categories = category_data.get("loss_categories", [])
        gain_categories = category_data.get("gain_categories", [])
        # Validate categories
        loss_categories = [cat for cat in loss_categories if isinstance(cat, str) and 1 <= len(cat.split()) <= 2]
        gain_categories = [cat for cat in gain_categories if isinstance(cat, str) and 1 <= len(cat.split()) <= 2]
        if len(loss_categories) < 5 or len(gain_categories) < 5:
            raise ValueError("Insufficient valid categories")
    except (json.JSONDecodeError, ValueError):
        # Fallback categories if LLM fails or returns invalid data
        loss_categories = ["Utility Payment", "Subscription Fee", "Online Purchase", "Rent Payment", "Grocery Shopping", "Insurance Bill"]
        gain_categories = ["Salary Deposit", "Tax Refund", "Gift Received", "Client Payment", "Investment Return", "Cash Deposit"]
    return loss_categories, gain_categories

# Cell 5: Generate transaction description using LLM
def generate_transaction_description(amount: float, category: str) -> dict:
    prompt = f"""
    Generate a bank transaction description (3-5 words, max 25 characters) for a transaction in the '{category}' category.
    Rules:
    - Use title case (e.g., 'Grocery Store Purchase').
    - No punctuation (commas, periods, etc.).
    - No parentheses, dashes, or dollar signs.
    - No amounts or numbers as words.
    - No typos.
    - Use simple, clear phrases.
    - Examples: 'Grocery Store Purchase', 'Utility Bill Payment', 'Salary Direct Deposit'
    """
    try:
        response = ollama.generate(model="mistral:7b-instruct-v0.3-q4_0", prompt=prompt)
        description = response['response'].strip()[:25]
    except:
        description = f"{category} Transaction"  # Fallback if LLM fails

    # Clean description
    description = description.replace("(", "").replace(")", "").replace(",", "").replace(":", "").replace("-", "").replace("$", "").replace(".", "")
    description = ' '.join(word.capitalize() for word in description.split())[:25]

    # Validate word count
    words = description.split()
    if len(words) < 3 or len(words) > 5:
        description = f"{category} Transaction"[:25]  # Fallback to generic description

    transaction = Transaction(
        description=description,
        category=category,
        amount=amount
    )
    return transaction.dict()

In [5]:

# Cell 6: Generate the bank statement
def generate_bank_statement(num_transactions: int = 10, account_holder: str = "John Doe") -> pd.DataFrame:
    # Get category lists
    loss_categories, gain_categories = generate_category_lists()

    # Generate random dates within the last 90 days
    start_date = datetime.now() - timedelta(days=90)
    dates = [start_date + timedelta(days=random.randint(0, 90)) for _ in range(num_transactions)]

    # Create transactions
    transactions = []
    for _ in range(num_transactions):
        is_gain = random.choice([True, False])
        category = random.choice(gain_categories if is_gain else loss_categories)
        amount = round(random.uniform(50, 1000), 2) if is_gain else round(random.uniform(-500, -10), 2)
        transaction = generate_transaction_description(amount, category)
        transactions.append(transaction)

    # Create DataFrame
    data = {
        "Date": [d.strftime("%Y-%m-%d") for d in dates],
        "Description": [t["description"] for t in transactions],
        "Category": [t["category"] for t in transactions],
        "Amount": [t["amount"] for t in transactions],
        "Balance": [0.0] * num_transactions,
        "Account Holder": [account_holder] * num_transactions,
        "Transaction ID": [fake.bban() for _ in range(num_transactions)]
    }
    df = pd.DataFrame(data)

    # Sort by date and calculate running balance
    df = df.sort_values("Date")
    initial_balance = 1000.0
    df["Balance"] = initial_balance + df["Amount"].cumsum()

    return df


In [6]:
# Cell 7: Create PDFs in different bank formats
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from datetime import datetime

# First Citizens Bank PDF Format (already updated in previous response)
def generate_pdf_first_citizens(df: pd.DataFrame, account_holder: str, output_filename: str) -> str:
    doc = SimpleDocTemplate(
        output_filename,
        pagesize=letter,
        rightMargin=0.5*inch,
        leftMargin=0.5*inch,
        topMargin=0.75*inch,
        bottomMargin=0.5*inch
    )
    elements = []

    styles = getSampleStyleSheet()
    
    bank_style = styles['Normal']
    bank_style.fontSize = 12
    bank_style.fontName = 'Helvetica-Bold'
    
    address_style = styles['Normal']
    address_style.fontSize = 10
    
    title_style = styles['Title']
    title_style.alignment = 1
    
    detail_style = styles['Normal']
    detail_style.fontSize = 10
    detail_style.leading = 12
    
    bank_name = Paragraph("FIRST CITIZENS BANK", bank_style)
    bank_address = Paragraph("231 Valley Farms Street<br/>Santa Monica, CA 90403<br/>firstcitizensbank@domain.com", address_style)
    statement_title = Paragraph("STATEMENT OF ACCOUNT", title_style)
    
    account_number = Paragraph(f"Account Number: 111-234-567-890", detail_style)
    statement_date = Paragraph(f"Statement Date: {datetime.now().strftime('%m/%d/%Y')}", detail_style)
    period_covered = Paragraph(f"Period Covered: {min(df['Date']).replace('-', '/')} to {max(df['Date']).replace('-', '/')} ", detail_style)
    page_number = Paragraph("Page 1 of 1", detail_style)
    
    customer_info = Paragraph(f"{account_holder}<br/>2450 Courage St, STE 108<br/>Brownsville, TX 78521", detail_style)
    
    initial_balance = 1000.0
    opening_balance = initial_balance
    total_credits = df[df['Amount'] > 0]['Amount'].sum()
    total_debits = abs(df[df['Amount'] < 0]['Amount'].sum())
    closing_balance = df['Balance'].iloc[-1]
    num_transactions = len(df)
    
    summary = [
        Paragraph(f"Opening Balance: {opening_balance:,.2f}", detail_style),
        Paragraph(f"Total Credit Amount: {total_credits:,.2f}", detail_style),
        Paragraph(f"Total Debit Amount: {total_debits:,.2f}", detail_style),
        Paragraph(f"Closing Balance: {closing_balance:,.2f}", detail_style),
        Paragraph("Account Type: Current Account", detail_style),
        Paragraph(f"Number of Transactions: {num_transactions}", detail_style),
    ]
    
    df = df.copy()
    df['Credit'] = df['Amount'].apply(lambda x: f"{x:,.2f}" if x > 0 else "")
    df['Debit'] = df['Amount'].apply(lambda x: f"{abs(x):,.2f}" if x < 0 else "")
    df['Balance'] = df['Balance'].apply(lambda x: f"{x:,.2f}")
    df['Date'] = df['Date'].apply(lambda x: x.replace('-', '/'))
    
    table_data = [["Date", "Description", "Credit", "Debit", "Balance"]]
    for _, row in df.iterrows():
        table_data.append([row['Date'], row['Description'], row['Credit'], row['Debit'], row['Balance']])
    
    page_width = letter[0] - (doc.leftMargin + doc.rightMargin)
    col_widths = [
        0.15 * page_width,
        0.35 * page_width,
        0.15 * page_width,
        0.15 * page_width,
        0.20 * page_width
    ]
    
    table = Table(table_data, colWidths=col_widths, splitByRow=True)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        ('ALIGN', (1, 1), (1, -1), 'LEFT'),
        ('ALIGN', (2, 1), (-1, -1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey]),
    ]))
    
    elements.append(bank_name)
    elements.append(bank_address)
    elements.append(Spacer(1, 0.1*inch))
    
    info_table_data = [
        [customer_info, statement_title],
        ["", account_number],
        ["", statement_date],
        ["", period_covered],
        ["", page_number]
    ]
    info_table = Table(info_table_data, colWidths=[2.5*inch, 4.5*inch])
    info_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
    ]))
    elements.append(info_table)
    elements.append(Spacer(1, 0.25*inch))
    
    for line in summary:
        elements.append(line)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("Transactions", styles['Heading2']))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(table)
    
    doc.build(elements)
    return output_filename


In [7]:

# Chase Bank PDF Format (Updated
def generate_pdf_chase(df: pd.DataFrame, account_holder: str, output_filename: str) -> str:
    doc = SimpleDocTemplate(
        output_filename,
        pagesize=letter,
        rightMargin=0.5*inch,
        leftMargin=0.5*inch,
        topMargin=0.75*inch,
        bottomMargin=0.5*inch
    )
    elements = []

    styles = getSampleStyleSheet()
    
    # Custom styles
    bank_style = styles['Normal']
    bank_style.fontSize = 12
    bank_style.fontName = 'Helvetica-Bold'
    
    detail_style = styles['Normal']
    detail_style.fontSize = 10
    detail_style.leading = 12
    
    contact_style = styles['Normal']
    contact_style.fontSize = 9
    contact_style.alignment = 2  # Right alignment
    
    heading_style = styles['Heading2']
    heading_style.fontName = 'Helvetica-Bold'
    
    # Header Section
    bank_name = Paragraph("JPMorgan Chase Bank, N.A.", bank_style)
    
    statement_period = Paragraph(
        f"{min(df['Date']).replace('-', '/')} through {max(df['Date']).replace('-', '/')} ",
        contact_style
    )
    account_number = Paragraph("Account Number: 987-654-321", contact_style)
    contact_info = Paragraph(
        "Customer Service: 1-800-935-9935<br/>TTY: 1-800-242-7383<br/>Para Español: 1-888-622-4273<br/>www.chase.com",
        contact_style
    )
    
    customer_info = Paragraph(
        f"{account_holder}<br/>123 Main St<br/>Anytown, USA 12345",
        detail_style
    )
    
    # Layout the header using a table to position elements
    header_data = [
        [bank_name, statement_period],
        ["", account_number],
        ["", contact_info]
    ]
    header_table = Table(header_data, colWidths=[3*inch, 4*inch])
    header_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
    ]))
    
    # Checking Summary Section
    initial_balance = 1000.0
    beginning_balance = initial_balance
    deposits = df[df['Amount'] > 0]
    withdrawals = df[df['Amount'] < 0]
    total_deposits = deposits['Amount'].sum()
    total_withdrawals = abs(withdrawals['Amount'].sum())
    ending_balance = df['Balance'].iloc[-1]
    
    summary_data = [
        ["Beginning Balance", f"${beginning_balance:,.2f}"],
        [f"Deposits and Additions ({len(deposits)})", f"${total_deposits:,.2f}"],
        [f"Electronic Withdrawals ({len(withdrawals)})", f"${total_withdrawals:,.2f}"],
        ["Ending Balance", f"${ending_balance:,.2f}"]
    ]
    
    summary_table = Table(summary_data, colWidths=[3*inch, 2*inch])
    summary_table.setStyle(TableStyle([
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
    ]))
    
    summary_note = Paragraph(
        "Your monthly service fee was waived because you maintained an average checking balance of $1,500 or a minimum...",
        detail_style
    )
    
    # Deposits and Additions Section
    deposits_data = [["Date", "Description", "Amount"]]
    for _, row in deposits.iterrows():
        deposits_data.append([
            row['Date'].replace('-', '/'),
            row['Description'],
            f"${row['Amount']:,.2f}"
        ])
    deposits_data.append(["", "TOTAL DEPOSITS", f"${total_deposits:,.2f}"])
    
    deposits_table = Table(deposits_data, colWidths=[1.5*inch, 3*inch, 1.5*inch])
    deposits_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        ('ALIGN', (1, 1), (1, -1), 'LEFT'),
        ('ALIGN', (2, 1), (2, -1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
        ('FONTNAME', (-1, -1), (-1, -1), 'Helvetica-Bold'),
    ]))
    
    # Electronic Withdrawals Section
    withdrawals_data = [["Date", "Description", "Amount"]]
    for _, row in withdrawals.iterrows():
        withdrawals_data.append([
            row['Date'].replace('-', '/'),
            row['Description'],
            f"${abs(row['Amount']):,.2f}"
        ])
    withdrawals_data.append(["", "TOTAL WITHDRAWALS", f"${total_withdrawals:,.2f}"])
    
    withdrawals_table = Table(withdrawals_data, colWidths=[1.5*inch, 3*inch, 1.5*inch])
    withdrawals_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        ('ALIGN', (1, 1), (1, -1), 'LEFT'),
        ('ALIGN', (2, 1), (2, -1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
        ('FONTNAME', (-1, -1), (-1, -1), 'Helvetica-Bold'),
    ]))
    
    # Daily Ending Balance Section
    daily_balance_data = [["Date", "Amount"]]
    # Add a few representative daily balances (e.g., first, middle, and last dates)
    unique_dates = df['Date'].unique()
    if len(unique_dates) > 0:
        daily_balance_data.append([unique_dates[0].replace('-', '/'), f"${df[df['Date'] == unique_dates[0]]['Balance'].iloc[-1]:,.2f}"])
        if len(unique_dates) > 2:
            mid_idx = len(unique_dates) // 2
            daily_balance_data.append([unique_dates[mid_idx].replace('-', '/'), f"${df[df['Date'] == unique_dates[mid_idx]]['Balance'].iloc[-1]:,.2f}"])
        daily_balance_data.append([unique_dates[-1].replace('-', '/'), f"${df[df['Date'] == unique_dates[-1]]['Balance'].iloc[-1]:,.2f}"])
    
    daily_balance_table = Table(daily_balance_data, colWidths=[1.5*inch, 1.5*inch])
    daily_balance_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        ('ALIGN', (1, 1), (1, -1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
    ]))
    
    # Assemble the PDF
    elements.append(header_table)
    elements.append(Spacer(1, 0.1*inch))
    elements.append(customer_info)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("CHECKING SUMMARY Chase Business Select Checking", heading_style))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(summary_table)
    elements.append(Spacer(1, 0.1*inch))
    elements.append(summary_note)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("DEPOSITS AND ADDITIONS", heading_style))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(deposits_table)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("ELECTRONIC WITHDRAWALS", heading_style))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(withdrawals_table)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("DAILY ENDING BALANCE", heading_style))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(daily_balance_table)
    
    doc.build(elements)
    return output_filename


In [8]:

# Wells Fargo Bank PDF Format
def generate_pdf_wellsfargo(df: pd.DataFrame, account_holder: str, output_filename: str) -> str:
    doc = SimpleDocTemplate(
        output_filename,
        pagesize=letter,
        rightMargin=0.5*inch,
        leftMargin=0.5*inch,
        topMargin=0.75*inch,
        bottomMargin=0.5*inch
    )
    elements = []

    styles = getSampleStyleSheet()
    
    # Custom styles
    account_type_style = styles['Normal']
    account_type_style.fontSize = 12
    account_type_style.fontName = 'Helvetica-Bold'
    
    detail_style = styles['Normal']
    detail_style.fontSize = 10
    detail_style.leading = 12
    
    contact_style = styles['Normal']
    contact_style.fontSize = 9
    contact_style.alignment = 2  # Right alignment
    
    heading_style = styles['Heading2']
    heading_style.fontName = 'Helvetica-Bold'
    
    bank_style = styles['Normal']
    bank_style.fontSize = 12
    bank_style.fontName = 'Helvetica-Bold'
    bank_style.alignment = 2  # Right alignment
    
    # Header Section
    account_type = Paragraph("Complete Advantage Checking", account_type_style)
    account_number = Paragraph("Account number: 970-962-9037", detail_style)
    statement_period = Paragraph(
        f"{min(df['Date']).replace('-', '/')} - {max(df['Date']).replace('-', '/')} ",
        contact_style
    )
    bank_name = Paragraph("WELLS FARGO", bank_style)
    
    customer_info = Paragraph(
        f"{account_holder}<br/>123 Main St<br/>Anytown, USA 12345",
        detail_style
    )
    
    contact_info = Paragraph(
        "QUESTIONS?<br/>Available by phone 24 hours a day, 7 days a week:<br/>"
        "1-800-CALL-WELLS (1-800-225-5935)<br/>"
        "TTY: 1-800-877-4833<br/>"
        "En español: 1-877-337-7464<br/>"
        "Online: wellsfargo.com<br/>"
        "Write: Wells Fargo Bank,<br/>420 Montgomery Street<br/>San Francisco, CA 94104",
        detail_style
    )
    contact_info_table = Table([[contact_info]], colWidths=[2.5*inch])
    contact_info_table.setStyle(TableStyle([
        ('BOX', (0, 0), (-1, -1), 1, colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'RIGHT'),
    ]))
    
    # Layout the header using a table
    header_data = [
        [account_type, bank_name],
        [account_number, statement_period],
        ["", contact_info_table]
    ]
    header_table = Table(header_data, colWidths=[4*inch, 3*inch])
    header_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
    ]))
    
    # Your Wells Fargo Section
    your_wells_fargo = Paragraph(
        "Your Wells Fargo<br/>"
        "It’s a great time to talk with a banker about how Wells Fargo’s accounts and services can help you compete day after day by saving you time and money. To find out how we can help, stop by any Wells Fargo location or call us at the number at the top of this statement.",
        detail_style
    )
    
    # Activity Summary Section
    initial_balance = 1000.0
    beginning_balance = initial_balance
    deposits = df[df['Amount'] > 0]
    withdrawals = df[df['Amount'] < 0]
    total_deposits = deposits['Amount'].sum()
    total_withdrawals = abs(withdrawals['Amount'].sum())
    ending_balance = df['Balance'].iloc[-1]
    
    summary_data = [
        ["Beginning balance on 02/01", f"${beginning_balance:,.2f}"],
        ["Deposits/Credits", f"${total_deposits:,.2f}"],
        ["Withdrawals/Debits", f"${total_withdrawals:,.2f}"],
        ["Ending Balance on 02/31", f"${ending_balance:,.2f}"]
    ]
    
    summary_table = Table(summary_data, colWidths=[3*inch, 1.5*inch])
    summary_table.setStyle(TableStyle([
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
    ]))
    
    account_details = Paragraph(
        f"Account number: 9709629037<br/>"
        f"{account_holder}<br/>"
        "For Direct Deposit and Automatic Payments use<br/>"
        "Routing Number (RTN): 053000219<br/>"
        "For Wire Transfer use<br/>"
        "Routing Number (RTN): 121000248",
        detail_style
    )
    
    activity_summary_layout = Table([[summary_table, account_details]], colWidths=[3*inch, 3*inch])
    activity_summary_layout.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ]))
    
    # Interest Summary Section
    interest_summary_data = [
        ["Interest paid this statement", "$0.25"],
        ["Average collected balance", "$2,977.87"],
        ["Annual percentage yield earned", "0.10%"],
        ["Interest earned this statement period", "$0.25"],
        ["Interest paid this year", "$0.05"]
    ]
    
    interest_summary_table = Table(interest_summary_data, colWidths=[3*inch, 1.5*inch])
    interest_summary_table.setStyle(TableStyle([
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
    ]))
    
    # Transaction History Section
    transactions_data = [["Date", "Description", "Deposits/Credits", "Withdrawals/Debits", "Ending daily balance"]]
    for _, row in df.iterrows():
        transactions_data.append([
            row['Date'].replace('-', '/'),
            row['Description'],
            f"${row['Amount']:,.2f}" if row['Amount'] > 0 else "",
            f"${abs(row['Amount']):,.2f}" if row['Amount'] < 0 else "",
            f"${row['Balance']:,.2f}"
        ])
    
    transactions_table = Table(transactions_data, colWidths=[1*inch, 2.5*inch, 1*inch, 1*inch, 1*inch])
    transactions_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        ('ALIGN', (1, 1), (1, -1), 'LEFT'),
        ('ALIGN', (2, 1), (-1, -1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
    ]))
    
    # Assemble the PDF
    elements.append(header_table)
    elements.append(Spacer(1, 0.1*inch))
    elements.append(customer_info)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("Your Wells Fargo", heading_style))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(your_wells_fargo)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("ACTIVITY SUMMARY", heading_style))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(activity_summary_layout)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("Interest summary", heading_style))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(interest_summary_table)
    elements.append(Spacer(1, 0.25*inch))
    
    elements.append(Paragraph("Transaction history", heading_style))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(transactions_table)
    
    doc.build(elements)
    return output_filename

In [ ]:

# Cell 8: Generate and Save Statements in All Formats
num_transactions = 12
account_holder = "John Doe"

statement = generate_bank_statement(num_transactions, account_holder)
print("Bank Statement Preview:")
print(statement.head(5))

csv_filename = f"version2.5/bank_statement_{account_holder.replace(' ', '_')}.csv"
statement.to_csv(csv_filename, index=False)
print(f"CSV saved as: {csv_filename}")

# Generate PDFs for each bank format
pdf_filename_first_citizens = f"version2.5/bank_statement_{account_holder.replace(' ', '_')}_first_citizens.pdf"
generate_pdf_first_citizens(statement, account_holder, pdf_filename_first_citizens)
print(f"First Citizens PDF saved as: {pdf_filename_first_citizens}")

pdf_filename_chase = f"version2.5/bank_statement_{account_holder.replace(' ', '_')}_chase.pdf"
generate_pdf_chase(statement, account_holder, pdf_filename_chase)
print(f"Chase PDF saved as: {pdf_filename_chase}")

pdf_filename_wellsfargo = f"version2.5/bank_statement_{account_holder.replace(' ', '_')}_wellsfargo.pdf"
generate_pdf_wellsfargo(statement, account_holder, pdf_filename_wellsfargo)
print(f"Wells Fargo PDF saved as: {pdf_filename_wellsfargo}")

C:\Users\cha\AppData\Local\Temp\ipykernel_16504\916404054.py:59: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return transaction.dict()


Bank Statement Preview:
          Date                Description            Category  Amount  \
2   2025-03-16   Electricity Bill Payment       Utility Bills -146.93   
6   2025-03-16   Electricity Bill Payment       Utility Bills -459.43   
4   2025-03-23   Subscription Service Fee       Subscriptions -463.87   
10  2025-04-07   Electricity Bill Payment       Utility Bills -477.40   
9   2025-04-09  Investment Returns Receiv  Investment Returns  681.96   

    Balance Account Holder      Transaction ID  
2    853.07       John Doe  AFIF75556025377006  
6    393.64       John Doe  VWFZ93343812408654  
4    -70.23       John Doe  JZDJ06657049455153  
10  -547.63       John Doe  VTKL90751525193395  
9    134.33       John Doe  DKMS92063600022343  


OSError: Cannot save file into a non-existent directory: 'testcopyfolder'